# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 ноября 2017, 23:59   
**Штраф за опоздание:** -2 балла после 23:59  4 декабря, -4 балла после 23:59 11 декабря, -6 баллов после 23:59 18 декабря

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw1.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

**Разбаловка:**   
За задание можно получить 10 баллов. Для этого нужно следующее:
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает    
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала качественнее, чем у дерева из sklearn

In [54]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

In [55]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=float('+inf'), sufficient_share=1.0, criterion='gini',
                 max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print 'invalid criterion name'
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print 'invalid max_features name'
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        # Ваш код:
        return l_s[0][0] + r_s[0][0] - (np.sum(l_c * l_c / l_s, axis = 1) + np.sum(r_c * r_c / r_s, axis = 1))
        
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        # Ваш код:
        eps = 0.0001
        log_priors_l = np.log2(np.clip(l_c / l_s, eps, 1 - eps))
        log_priors_r = np.log2(np.clip(r_c / r_s, eps, 1 - eps))
        return -(np.sum(l_c * log_priors_l, axis = 1) + np.sum(r_c * log_priors_r, axis = 1))

    def __misclass(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        # Ваш код:
        return 1 - (np.amax(l_c, axis = 1) + np.amax(r_c, axis = 1)) / (l_s[0][0] + r_s[0][0])

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        # Ваш код:
        return feature_ids[:int(np.sqrt(n_feature))]
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        # Ваш код:
        return feature_ids[:int(np.log2(n_feature))]

    def __get_feature_ids_N(self, n_feature):
        # Ваш код:
        return np.arange(n_feature, dtype = int)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, int(feature_id)] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода? Получаем столбец признака таблицы - x, сортируем его и y. Далее 
        # записываем число уникальных классов вектора y.
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = np.unique(y).shape[0]
        
        # Что делает этот блок кода? В первой строчки отрезаем по бокам отсортированного вектора концы,
        # чтобы далее проверить больше ли в векторе элементов чем min_samples_split. Во-второй строчке 
        # создается вектор с аргументами в которых меняется значение кластера с одного на другой.
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)
        
        if len(r_border_ids) == 0:
            return float('+inf'), None
        
        # Что делает этот блок кода? В первой строчке строится вектор состоящий из длин последовательностей  
        # одинаковых меток. В следующих строках кода сначала создается матрица в которую для каждой 
        # строки(т.е. разбиения) записывается длина вхождений того или иного класса.
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1]) 
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:self.min_samples_split], minlength=class_number)
        
        # Что делает этот блок кода? Для различных вариантов разбиений строятся матрицы в строках которых
        # записывается количество тех или иных меток классов левой и правой части разбиения соответственно. 
        # Так же строятся матрицы l_sizes и r_sizes в которых в строках записывается число элементов
        # в правой и левой частях разбиения. Далее в зависимости от выбора оценочной функции выдается вектор 
        # со всеми значениями оценочной функции разбиений и ищется минимум из этих значений. 
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(y) - l_class_count 
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)
    
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код:
        
        # Необходимо использовать следующее:
        # self.LEAF_TYPE +
        # self.NON_LEAF_TYPE +

        # self.tree +
        # self.max_depth +
        # self.sufficient_share +
        # self.min_samples_split +

        # self.get_feature_ids +
        # self.__find_threshold +
        # self.__div_samples +
        # self.__fit_node +
        
        if depth >= self.max_depth: # отсекаем по глубине
            max_count_class_id = np.bincount(y, minlength = np.unique(y).shape[0]).argmax()
            self.tree[node_id] = np.array([self.LEAF_TYPE, max_count_class_id, None])
            return  
            
        if y.shape[0] < self.min_samples_split: # отсекаем по min_samples_split
            max_count_class_id = np.bincount(y, minlength = np.unique(y).shape[0]).argmax()
            self.tree[node_id] = np.array([self.LEAF_TYPE, max_count_class_id, None])
            return 
        
        # отсекаем по sufficient_share
        if np.max(np.bincount(y, minlength = np.unique(y).shape[0]))/y.shape[0]  >= self.sufficient_share:
            max_count_class_id = np.bincount(y, minlength = np.unique(y).shape[0]).argmax()
            self.tree[node_id] = np.array([self.LEAF_TYPE, max_count_class_id, None])
            return 
            
        ids_f = self.get_feature_ids(x.shape[1]) # перебираем все признаки и ищем лучший threshold
        split_threshold = np.array([ np.array(self.__find_threshold(x[:, id_f], y)) for id_f in ids_f])
        best_split_f_id = split_threshold[:, 0].argmin()
        
        if split_threshold[best_split_f_id][0] == float("+inf"): # проверка на случай когда все признаки 
            max_count_class_id = np.bincount(y, minlength = np.unique(y).shape[0]).argmax()
            self.tree[node_id] = np.array([self.LEAF_TYPE, max_count_class_id, None])
            return 
                
        # строим node
        self.tree[node_id] = np.array([0,ids_f[best_split_f_id],split_threshold[best_split_f_id][1]])
        
        # делим таблицу в соответствии с threshold
        x_left, x_right,y_left, y_right = self.__div_samples(x,y,self.tree[node_id][1],self.tree[node_id][2])
        
        if y_left.shape[0] == 0 or y_right.shape[0] == 0:
            max_count_class_id = np.bincount(y, minlength = np.unique(y).shape[0]).argmax()
            self.tree[node_id] = np.array([self.LEAF_TYPE, max_count_class_id, None])
            return 
        
        self.__fit_node(x_left, y_left, 2*node_id + 1, depth + 1)   # рекурсивно строим дерево
        self.__fit_node(x_right, y_right, 2*node_id + 2, depth + 1) # для каждой ветви
        pass
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[int(feature_id)] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[int(feature_id)] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [56]:
df = pd.read_csv('./cs-training.csv', sep=',').dropna()
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [57]:
x = df.as_matrix(columns=df.columns[1:])
y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0])

In [58]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы

In [59]:
t1 = time()
my_clf.fit(x, y)
t2 = time()
print(t2 - t1)

t1 = time()
clf.fit(x, y)
t2 = time()
print(t2 - t1)

0.886554002762
2.28022623062


## Проверка качества работы

In [60]:
gkf = KFold(n_splits=5, shuffle=True)

In [61]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))

0.933441423464
0.93132119398
0.934439178515
0.932277375904
0.930985739825


In [62]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

0.891410991935
0.891577284443
0.895318865885
0.892159308223
0.893485220139
